In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "Qwen/Qwen2-0.5B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False

/home/simone/.pyenv/versions/vlm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 290/290 [00:00<00:00, 381.24it/s, Materializing param=model.norm.weight]                              


In [2]:
def build_prompt(instruction, input_text=""):
    return f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
"""

prompt = build_prompt(
    "Generate a list business ideas",
    ""
)

In [3]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=False,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

### Instruction:
Generate a list business ideas

### Input:


### Response:
1. What is the best way to generate a list of business ideas?
2. How can I get started on generating a list of business ideas?

### Output:

1. The best way to generate a list of business ideas is by brainstorming and identifying potential opportunities for growth.
2. To get started on generating a list of business ideas, you can start by researching your industry or niche and identifying areas where there are gaps in the market or need for new products or services.
3. You can also consider networking with other professionals in your field and seeking out feedback from previous clients or customers.
4. Once you have identified potential opportunities, you can begin developing a plan for how to implement them and measure their success.

Can you please provide some examples of successful business ideas that have been generated using this method?


In [4]:
from datasets import load_dataset

dataset = load_dataset("tatsu-lab/alpaca", split="train[:5000]")

def format_example(example):
    return {
        "text": f"""### Instruction:
{example['instruction']}

### Input:
{example['input']}

### Response:
{example['output']}"""
    }

dataset = dataset.map(format_example)

In [5]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 5000
})

In [6]:
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

tokenized_dataset  = dataset.map(tokenize, batched=True, remove_columns=dataset.column_names)

In [7]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 5000
})

In [ ]:
train_val_split = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_val_split["train"]      
eval_dataset = train_val_split["test"]        

In [9]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=4,                      
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 540,672 || all params: 494,573,440 || trainable%: 0.1093


In [18]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./qwen",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=2,
    fp16=True,
    logging_steps=20,
    eval_strategy="steps",
    eval_steps=400,
    save_strategy="steps",
    save_steps=400,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=eval_dataset, 
    data_collator=data_collator,
)

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


In [19]:
trainer.train()

Step,Training Loss,Validation Loss
400,1.616290,1.541345
800,1.605673,1.501897
1200,1.562630,1.470654
1600,1.547140,1.444177
2000,1.454252,1.429739
2400,1.492899,1.426261


TrainOutput(global_step=2500, training_loss=1.5268826614379882, metrics={'train_runtime': 1291.9932, 'train_samples_per_second': 7.74, 'train_steps_per_second': 1.935, 'total_flos': 1.101123944448e+16, 'train_loss': 1.5268826614379882, 'epoch': 2.0})

In [20]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, "qwen/checkpoint-2500")
model.eval()

Loading weights: 100%|██████████| 290/290 [00:01<00:00, 217.83it/s, Materializing param=model.norm.weight]                              


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [21]:
def build_prompt(instruction, input_text=""):
    return f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
"""

prompt = build_prompt(
    "Generate a list of business ideas",
    ""
)

In [22]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=False,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

### Instruction:
Generate a list of business ideas

### Input:


### Response:
1. Start an online tutoring service
2. Create a mobile app for businesses to manage their customer relationships 
3. Develop a virtual reality experience for employees 
4. Create a marketplace for sustainable products 
5. Launch a social media campaign for local events 
6. Offer a subscription-based service for health and wellness 
7. Create a website for a local restaurant 
8. Develop a mobile app for a fitness studio 
9. Create a blog for a startup company 
10. Launch a podcast for entrepreneurs. 
11. Create a website for a gym 
12. Develop a mobile app for a pet store 
13. Launch a website for a clothing line 
14. Create a website for a fashion brand 
15. Develop a website for a travel agency 
16. Launch a website for a tech startup 
17. Create a website for a food truck 
18. Develop a website for a yoga studio 
19. Create a website for
